### 导入矢量图层 

#### 导入要用到的包

In [1]:
from qgis.core import * 
from qgis.core import QgsProject
from qgis.core import (QgsVectorLayer)

#### 初始化QGIS

In [2]:
#提供qgis安装路径
QgsApplication.setPrefixPath('/opt/anaconda3/envs/qgis3/QGIS.app/Contents/MacOS', True)

# 创建对QgsApplication的引用，第二个参数设置为False将禁用GUI
qgs = QgsApplication([], False)

# 加载数据提供者和图层注册
qgs.initQgis()

#### 加载项目

In [3]:
# 获取项目实例
project = QgsProject.instance()

# 将项目保存到新文件
project.write('/Users/xin-yi.song/Desktop/苏州公安-公共自行车/P5：站点选址支持（盲区识别）/qgis_test_17.qgz')

True

#### 加载图层

In [4]:
#获取路网文件的路径
lpath = '/Users/xin-yi.song/Desktop/苏州公安-公共自行车/P5：站点选址支持（盲区识别）/路网_utf8.geojson'

#构造QgsVectorLayer实例,格式为vlayer = QgsVectorLayer(data_source, layer_name, provider_name)
llayer = QgsVectorLayer(lpath, '路网_utf8', 'ogr')

#使用addMapLayer()方法将图层添加到当前项目
QgsProject.instance().addMapLayer(llayer)

In [5]:
#获取自行车站点文件的路径
zpath = '/Users/xin-yi.song/Desktop/苏州公安-公共自行车/data_from_mengyin/sta_info/sta_info_gcj.geojson'

#构造QgsVectorLayer实例,格式为vlayer = QgsVectorLayer(data_source, layer_name, provider_name)
zlayer = QgsVectorLayer(zpath, '自行车站点_gcj', 'ogr')

#使用addMapLayer()方法将图层添加到当前项目
QgsProject.instance().addMapLayer(zlayer)

In [6]:
#获取已加载图层和图层ID的列表，使用mapLayers()方法
QgsProject.instance().mapLayers()

{'自行车站点_gcj_e0c65c3a_c31a_470a_91f8_02f9da7cca6b': <qgis._core.QgsVectorLayer at 0x1200f0730>,
 '路网_utf8_e09948c6_6ea8_4df5_bc6e_ac301c904e26': <qgis._core.QgsVectorLayer at 0x1200f0a50>}

In [7]:
# 将项目保存到新文件
project.write('/Users/xin-yi.song/Desktop/苏州公安-公共自行车/P5：站点选址支持（盲区识别）/qgis_test_17.qgz')

True

### 创建图

#### 导入要用的包

In [8]:
from qgis.analysis import *

#### 检索图层字段信息

In [9]:
# “layer”是一个QgsVectorLayer实例
layer = llayer

#可以通过调用QgsVectorLayer对象的fields()方法检索一个矢量图层相关字段的信息
n = 0
for field in layer.fields():
    print(n, field.name(), field.typeName())
    n = n+1

0 link_id Integer
1 segment_id String
2 road_id String
3 roadname String
4 fnode String
5 tnode String
6 length String
7 road_class String
8 p_lanes String
9 n_lanes String
10 max_speed String
11 direction String
12 ad_code String
13 toll_flag String
14 status String
15 link_type String
16 fc String
17 u_line String
18 over_head String
19 ramp String
20 mesh String
21 forwardroa String
22 reverseroa String
23 link_flag Integer
24 err_flag Integer
25 gd_id String
26 ge_id String
27 gj_id String
28 gm_id String
29 gjjdd_id String
30 gjjzd_id String
31 gjj_id String
32 zone_id String


In [10]:
#查询要素数量
print("features:", layer.featureCount())

features: 94138


#### 构造director

In [11]:
#构造一个director需要传入一个矢量图层，它可以作为图结构和各路段允许移动方向（单行，双行；正向，反向）等信息的数据源
#方法的调用长这个样子
# director = QgsVectorLayerDirector
# (
#  vl, #用来创建图的矢量图层
#  directionFieldId, #属性表字段编号，其中储存道路方向的信息。如果为-1，则不用这里面的信息。是一个整型。
#  directDirectionValue, #道路正方向的字段值（从第一个线段点向最后一个移动）。是一个字符串。
#  reverseDirectionValue, #道路反方向的字段值（从最后一个线段点向第一个移动）。是一个字符串。
#  bothDirectionValue, #双向道路的字段值（对这种道路能从第一个线段点向最后一个移动，也能从最后一个向第一个移动）。是一个字符串。
#  defaultDirection #默认道路方向。这个值用于directionFieldId未设置字段的道路或具有不同于上述三个值中的任何一个值的道路。这个值是一个整型。1表示正方向，2表示反方向，3表示双向。
# )

In [12]:
#根据路网_utf8构造director
director = QgsVectorLayerDirector(layer, 11, '2', '3', '1', 3)

#利用QgsDistanceStrategy策略来计算边的特性，它考虑了路线的长度
strategy = QgsNetworkDistanceStrategy()

#把这个策略传递给director
director.addStrategy(strategy)

#### 使用builder创建图

In [13]:
# QgsGraphBuilder 类构建器需要几个参数：
# crs — 使用的坐标参照系统,强制性参数。路网和自行车站点均使用gcj坐标。
# otfEnabled — 是否使用动态投影。默认为是（使用动态投影）
# topologyTolerance — 拓扑容差。默认值为0.
# ellipsoidID — 使用的椭球体。默认为“WGS84”.

In [14]:
# 只设定坐标参照系统，其他值则为默认。
myCRS =  QgsCoordinateReferenceSystem(4326)
builder = QgsGraphBuilder(myCRS)

In [15]:
#定义将被绑定的点
#对每个额外的点将会匹配最近的图顶点或最近的图边
#后面一种情况中边会被分割，并且加入一个新的顶点
startPoint = QgsPointXY(120.6, 31.3) 

In [16]:
#构建图，并把点绑定上去
#tiedPoints是一个由绑定点的坐标构成的list
#构建图需要花费一些时间（由一个图层中要素的数量和要素大小决定）
tiedPoints = director.makeGraph(builder, [startPoint])

In [17]:
#当构建操作完成后我们就能得到图并用于分析
graph = builder.graph()

In [18]:
#用下边的代码我们可以获得绑定点的顶点编号。
startId = graph.findVertex(tiedPoints[0])
print(startId)

251647


### 图分析

#### Dijkstra算法

In [19]:
#网络分析用于寻找两个问题的答案：哪些顶点是相连的，和如何寻找最短路径。要解决这些问题，网络分析库提供Dijkstra算法。

#Dijkstra算法寻找从图的其中一个顶点到其他所有顶点的最短路径，和最优化的参数值。结果可以作为最短路径树来呈现。

# 最短路径树是有向带权图（或更精确的说是树），其有如下特性：
# 只有一个顶点而没有入边——树的根结点
# 其他所有顶点都只有一个入边
# 如果顶点B可从顶点A到达，那么从A到B是单向可达边并且在此图中是最佳（最短）的

#用QgsGraphAnalyzer类的shortestTree和dijkstra方法来获得最短路径树。推荐使用dijkstra方法，因为它更快且在使用内存上更高效。
#(tree, cost) = QgsGraphAnalyzer.dijkstra(graph, startId, 0)
# 数据源——输入的图
# 起始顶点ID——树结点的编号（树的根结点）
# 度量值——使用的边属性的编号（从0开始）

#dijkstra方法返回两个数组。
#第一个数组的元素i存储了入边的编号，或者如果没有入边，则存储为-1。
#第二个数组的元素i存储的是从树根结点到顶点i的距离，或如果顶点i不能从根结点到达，则存储为DOUBLE_MAX

In [20]:
from qgis.analysis import *

layer = llayer
director = QgsVectorLayerDirector(layer, 11, '2', '3', '1', 3)
strategy = QgsNetworkDistanceStrategy()
director.addStrategy(strategy)
myCRS =  QgsCoordinateReferenceSystem(4326)
builder = QgsGraphBuilder(myCRS)

pStart = QgsPointXY(120.6, 31.3)
tiedPoint = director.makeGraph(builder, [pStart])
pStart = tiedPoint[0]

graph = builder.graph()

idStart = graph.findVertex(pStart)

(tree, costs) = QgsGraphAnalyzer.dijkstra(graph, idStart, 0)

In [21]:
print(len(tree))

251648


#### 寻找最短路径

In [22]:
#下面是找到两点之间最优路径的方法:
#当图被创建时，两个点（起点A和终点B）都绑定在了图上。
#然后使用dijkstra()算法，我们创建了根在起始点A的最短路径树。
#在同一棵树中我们还找到了终点B，并开始从点B向A逐个结点地访问这棵树。

# 整个算法可以写作：
# assign Т = B
# while Т != A
#     add point Т to path
#     get incoming edge for point Т
#     look for point ТТ, that is start point of this edge
#     assign Т = ТТ
# add point А to path

In [92]:
from qgis.analysis import *

layer = llayer
director = QgsVectorLayerDirector(layer, 11, '2', '3', '1', 3)
strategy = QgsNetworkDistanceStrategy()
director.addStrategy(strategy)
myCRS =  QgsCoordinateReferenceSystem(4326)
builder = QgsGraphBuilder(myCRS)

pStart = QgsPointXY(120.6, 31.3)
pStop = QgsPointXY(120.53,31.27)

tiedPoints = director.makeGraph(builder, [pStart, pStop])
graph = builder.graph()

tStart = tiedPoints[0]
tStop = tiedPoints[1]

idStart = graph.findVertex(tStart)
idStop = graph.findVertex(tStop)

(tree, cost) = QgsGraphAnalyzer.dijkstra(graph, idStart, 0)

In [94]:
print(len(tree))

251649


In [45]:
if tree[idStop] == -1:
  print("Path not found")
else:
  p = []
  curPos = idStop
  while curPos != idStart:
    p.append(graph.vertex(graph.edge(tree[curPos]).toVertex()).point())
    curPos = graph.edge(tree[curPos]).fromVertex();
    
  p.append(tStart)

In [60]:
print(len(p))

227


In [95]:
#计算两点的直线距离
from qgis.core import QgsDistanceArea

d = QgsDistanceArea()
d.setEllipsoid('WGS84')

print("Distance in meters: ", d.measureLine(pStart, pStop))

Distance in meters:  7449.193059671591


#### 保存最短路

In [62]:
#导入要用的包
from qgis.PyQt.QtCore import QVariant

In [63]:
#定义字段
fields_new = QgsFields()
fields_new.append(QgsField("ID", QVariant.Int))

True

In [73]:
#创建矢量文件
path_new = '/Users/xin-yi.song/Desktop/苏州公安-公共自行车/P5：站点选址支持（盲区识别）/最短路.geojson'
writer = QgsVectorFileWriter(path_new, "UTF-8", fields_new, QgsWkbTypes.LineString, driverName="GeoJSON")

if writer.hasError() != QgsVectorFileWriter.NoError:
    print("Error when creating GeoJSON: ",  w.errorMessage())

In [74]:
#将QgsPointXY转换成QgsPoint
for i in range(0,len(p)):
    p[i] = QgsPoint(p[i])

#添加要素
fet = QgsFeature()
fet.setGeometry(QgsGeometry.fromPolyline(p))
fet.setAttributes([1])
writer.addFeature(fet)

# 删除writer以将要素保存到磁盘
del writer

In [75]:
# 显示一些统计
vl = QgsVectorLayer(path_new, '最短路', 'ogr')
pr = vl.dataProvider()

print("fields:", len(pr.fields()))
print("features:", pr.featureCount())
e = vl.extent()
print("extent:", e.xMinimum(), e.yMinimum(), e.xMaximum(), e.yMaximum())

# 遍历要素
features = vl.getFeatures()
for fet in features:
    print("F:", fet.id(), fet.attributes(), fet.geometry().asPolyline())

fields: 1
features: 1
extent: 120.52999153496289 31.262287 120.60016152525336 31.2988996092115
F: 0 [1] [<QgsPointXY: POINT(120.52999153496288898 31.26999951489758445)>, <QgsPointXY: POINT(120.53000400000000525 31.2697819999999993)>, <QgsPointXY: POINT(120.53005600000000186 31.26916699999999949)>, <QgsPointXY: POINT(120.53020399999999768 31.26793100000000081)>, <QgsPointXY: POINT(120.53020800000000179 31.26789700000000138)>, <QgsPointXY: POINT(120.53032899999999472 31.26688199999999895)>, <QgsPointXY: POINT(120.53047700000000475 31.26670700000000025)>, <QgsPointXY: POINT(120.53060100000000432 31.26574899999999957)>, <QgsPointXY: POINT(120.53079099999999357 31.26368300000000033)>, <QgsPointXY: POINT(120.53074499999999603 31.26358300000000057)>, <QgsPointXY: POINT(120.53083800000000281 31.2622870000000006)>, <QgsPointXY: POINT(120.53286900000000514 31.26232900000000114)>, <QgsPointXY: POINT(120.5344390000000061 31.26242200000000082)>, <QgsPointXY: POINT(120.53500800000000481 31.262454999

#### 可达区域

In [105]:
from qgis.analysis import *

layer = llayer
director = QgsVectorLayerDirector(layer, 11, '2', '3', '1', 3)
strategy = QgsNetworkDistanceStrategy()
director.addStrategy(strategy)
myCRS =  QgsCoordinateReferenceSystem("EPSG:4326")
builder = QgsGraphBuilder(myCRS)

pStart = QgsPointXY(120.56646, 31.28188)

tiedPoints = director.makeGraph(builder, [pStart])
tStart = tiedPoints[0]

graph = builder.graph()
idStart = graph.findVertex(tStart)

(tree, cost) = QgsGraphAnalyzer.dijkstra(graph, idStart, 0)

In [106]:
print(len(cost))
print([x for x in cost if x<200])

251648
[149.11324939707936, 159.01418614000102, 189.98847879158424, 165.90749009361232, 181.006711330923, 28.193943639993787, 57.42985004211654, 91.19728783344951, 144.87101064138955, 0.0]


In [110]:
upperBound = []
upperBound_Points = []

r = 200
i = 0
while i < len(cost):
  if cost[i] >= r and tree[i] != -1:
    fromVertexId = graph.edge(tree[i]).fromVertex()
    toVertex = graph.vertex(graph.edge(tree[i]).toVertex()).point()
    
    if cost[fromVertexId] <= r:
      upperBound.append(i) 
      upperBound_Points.append(toVertex)
      
  i = i + 1

print(upperBound)
print(upperBound_Points)

[14151, 148914, 164899, 171583]
[<QgsPointXY: POINT(120.5685579999999959 31.28248699999999971)>, <QgsPointXY: POINT(120.56774099999999805 31.28313899999999848)>, <QgsPointXY: POINT(120.56654199999999832 31.28224799999999917)>, <QgsPointXY: POINT(120.56838199999999972 31.28082699999999861)>]


#### 保存可达区域

In [124]:
#导入要用的包
from qgis.PyQt.QtCore import QVariant

In [125]:
#定义字段
fields_new = QgsFields()
fields_new.append(QgsField("ID", QVariant.Int))

True

In [126]:
#创建矢量文件
path_new = '/Users/xin-yi.song/Desktop/苏州公安-公共自行车/P5：站点选址支持（盲区识别）/可达区域.geojson'
writer = QgsVectorFileWriter(path_new, "UTF-8", fields_new, QgsWkbTypes.Polygon, driverName="GeoJSON")

if writer.hasError() != QgsVectorFileWriter.NoError:
    print("Error when creating GeoJSON: ",  w.errorMessage())

In [129]:
#添加要素
fet = QgsFeature()
fet.setGeometry(QgsGeometry.fromPolygonXY([upperBound_Points]))
fet.setAttributes([1])
writer.addFeature(fet)

# 删除writer以将要素保存到磁盘
del writer

In [130]:
# 显示一些统计
vl = QgsVectorLayer(path_new, '可达区域', 'ogr')
pr = vl.dataProvider()

print("fields:", len(pr.fields()))
print("features:", pr.featureCount())
e = vl.extent()
print("extent:", e.xMinimum(), e.yMinimum(), e.xMaximum(), e.yMaximum())

# 遍历要素
features = vl.getFeatures()
for fet in features:
    print("F:", fet.id(), fet.attributes(), fet.geometry().asPolygon())

fields: 1
features: 1
extent: 120.566542 31.280827 120.568558 31.283139
F: 0 [1] [[<QgsPointXY: POINT(120.5685579999999959 31.28248699999999971)>, <QgsPointXY: POINT(120.56774099999999805 31.28313899999999848)>, <QgsPointXY: POINT(120.56654199999999832 31.28224799999999917)>, <QgsPointXY: POINT(120.56838199999999972 31.28082699999999861)>, <QgsPointXY: POINT(120.5685579999999959 31.28248699999999971)>]]
